# Comprehensive Polymarket Election Prediction Analysis
====================================================

This notebook integrates results from:
- Feature importance analysis
- Feature relationship analysis
- Prediction error analysis
- Actionable insights development

In [ ]:
# Set plotting style - use a current style
plt.style.use('default')  # Use default style
sns.set_theme(style="whitegrid")  # Use whitegrid from current seaborn
sns.set_context("paper", font_scale=1.2)

# Create output directory for combined results
output_dir = "combined_analysis"
os.makedirs(output_dir, exist_ok=True)

# Load core datasets
df = pd.read_csv("modified_analysis/cleaned_election_data.csv")
feature_importance = pd.read_csv("feature_importance_results/consensus_feature_ranking.csv")

# Try to load error patterns, but handle case if file doesn't exist
try:
    error_patterns = pd.read_csv("prediction_error_analysis/error_patterns_by_feature_bin.csv")
except FileNotFoundError:
    error_patterns = pd.DataFrame()
    print("Error patterns file not found. Will skip that analysis.")

# Try to load thresholds, but handle case if file doesn't exist
try:
    thresholds = pd.read_csv("actionable_insights/market_quality_thresholds.csv")
except FileNotFoundError:
    thresholds = pd.DataFrame()
    print("Thresholds file not found. Will use correlation data instead.")
    # Create a simple version based on correlations if needed
    if 'feature' in feature_importance.columns:
        thresholds = pd.DataFrame({
            'feature': feature_importance['feature'].head(10),
            'direction': ["Higher is better"] * 10,  # Will be fixed later
            'suggested_threshold': [0] * 10  # Will be calculated later
        })

# Define top features based on consensus ranking or fall back to important columns
top_features = []
if 'feature' in feature_importance.columns:
    top_features = feature_importance['feature'].head(10).tolist()
else:
    print("Feature importance file doesn't have expected format.")
    # Fall back to common important features from your previous results
    top_features = [
        'price_range', 'unique_traders_count', 'price_fluctuations', 
        'volumeNum', 'final_week_momentum', 'buy_sell_ratio',
        'event_commentCount', 'price_volatility', 'volume_acceleration', 
        'trader_concentration'
    ]
    # Filter to only include columns that exist in the dataset
    top_features = [f for f in top_features if f in df.columns]


OSError: 'whitegrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

# 1. Key Findings Summary

In [ ]:
# Load and display key metrics
display(Markdown("### Model Performance"))
model_performance = pd.read_csv("feature_importance_results/model_performance_comparison.csv")
display(model_performance)

# Create executive summary table
display(Markdown("### Top Predictive Features"))
top_features_summary = feature_importance.head(10).copy()

# Enrich with directional relationship to accuracy
for idx, row in top_features_summary.iterrows():
    feature = row['feature']
    corr = df[['brier_score', feature]].corr().iloc[0, 1]
    relationship = "Higher values → worse predictions" if corr > 0 else "Lower values → worse predictions"
    top_features_summary.loc[idx, 'relationship_to_accuracy'] = relationship
    top_features_summary.loc[idx, 'correlation_with_brier'] = corr

display(top_features_summary[['feature', 'importance_score', 'relationship_to_accuracy', 'correlation_with_brier']])


# 2. Integrated Feature Analysis 


In [ ]:
# Create correlation matrix for top features + brier score
correlation_df = df[top_features + ['brier_score']].corr()

# Plot correlation matrix
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_df, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(correlation_df, mask=mask, cmap=cmap, vmax=.5, vmin=-.5, 
            square=True, linewidths=.5, annot=True, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.savefig(f"{output_dir}/integrated_correlation_matrix.png", dpi=300)
plt.show()

## 3. Feature-Error Relationship Analysis"

In [ ]:
# Load error data
largest_errors = pd.read_csv("prediction_error_analysis/largest_error_markets.csv")

display(Markdown("### Markets with Largest Prediction Errors"))
display(largest_errors[['question', 'actual_brier', 'predicted_brier', 'error']].head())

# Integrate feature values with error analysis
# Load the test data with feature values
X_test_orig = pd.read_csv("modified_analysis/X_test_original.csv")

# Create integrated error analysis that combines error magnitude with feature values
display(Markdown("### Error Patterns by Feature Values"))

# Select 2-3 most important features for detailed error analysis
key_features = top_features[:3]

fig, axes = plt.subplots(len(key_features), 1, figsize=(10, 4*len(key_features)))

for i, feature in enumerate(key_features):
    if feature in X_test_orig.columns:
        # Load error data from prediction error analysis
        # In a real notebook, you might need to recreate this from your model results
        error_feature_df = pd.read_csv(f"prediction_error_analysis/error_by_{feature}.csv", 
                                      error_bad_lines=False) if os.path.exists(f"prediction_error_analysis/error_by_{feature}.csv") else None
        
        if error_feature_df is not None:
            sns.scatterplot(x=feature, y='abs_error', data=error_feature_df, ax=axes[i])
            sns.regplot(x=feature, y='abs_error', data=error_feature_df, 
                      scatter=False, line_kws={"color": "red"}, ax=axes[i])
            axes[i].set_title(f'Error Magnitude by {feature}')

plt.tight_layout()
plt.savefig(f"{output_dir}/integrated_error_analysis.png", dpi=300)
plt.show()

## 4. Actionable Insights Framework

In [ ]:
# Create a table showing the thresholds for good prediction quality
display(Markdown("### Market Quality Indicators"))
display(thresholds)

# Generate a scoring example
display(Markdown("### Practical Application Example"))

# Calculate a quality score for each market
# Create scoring function based on thresholds
def calculate_quality_score(market_data, threshold_data):
    score = 0
    reasons = []
    
    for _, row in threshold_data.iterrows():
        feature = row['feature']
        threshold = row['suggested_threshold']
        direction = row['direction']
        
        if feature in market_data:
            value = market_data[feature]
            
            if direction == "Lower is better" and value <= threshold:
                score += 1
                reasons.append(f"{feature}: {value:.4f} ≤ {threshold:.4f} ✓")
            elif direction == "Higher is better" and value >= threshold:
                score += 1
                reasons.append(f"{feature}: {value:.4f} ≥ {threshold:.4f} ✓")
            else:
                if direction == "Lower is better":
                    reasons.append(f"{feature}: {value:.4f} > {threshold:.4f} ✗")
                else:
                    reasons.append(f"{feature}: {value:.4f} < {threshold:.4f} ✗")
    
    return score, reasons

# Select a few example markets with different Brier scores
example_markets = df.sort_values('brier_score').iloc[[0, len(df)//4, len(df)//2, 3*len(df)//4, -1]].copy()

# Calculate quality score for each example
for idx, market in example_markets.iterrows():
    score, reasons = calculate_quality_score(market, thresholds)
    example_markets.loc[idx, 'quality_score'] = score
    example_markets.loc[idx, 'quality_assessment'] = '; '.join(reasons[:3]) + "..." if len(reasons) > 3 else '; '.join(reasons)

display(example_markets[['question', 'brier_score', 'quality_score', 'quality_assessment']])

# Save the consolidated analysis
plt.tight_layout()
